In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from transformers import ElectraForPreTraining, ElectraTokenizerFast
import torch

In [ ]:
model = ElectraForPreTraining.from_pretrained('Seznam/small-e-czech')

In [30]:
model

ElectraForPreTraining(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=25

In [7]:
tokenizer = ElectraTokenizerFast.from_pretrained("Seznam/small-e-czech")

In [8]:
import pickle

In [9]:
data = None
with open('stenozaznamy_psp_sample.pickle', 'rb') as picklefile:
    data = pickle.load(picklefile)

In [ ]:
data

In [11]:
data = [datapoint['text'] for datapoint in data['results']]

In [12]:
data = data[0:100]

In [ ]:
data[0]

In [51]:
query_tokens = torch.tensor(tokenizer.encode('Rusko válka Ukrajina'))
values_tokens = [tokenizer.encode(value) for value in data]

In [17]:
query_tokens

[2, 10127, 10026, 28174, 3]

In [19]:
encoded_query = model.electra.encoder(model.electra.embeddings_project(model.electra.embeddings(torch.tensor(query_tokens).unsqueeze(0))))

In [33]:
query = model.electra.embeddings_project(model.electra.embeddings(torch.tensor(query_tokens).unsqueeze(0)))

In [28]:
value = model.electra.embeddings_project(model.electra.embeddings(torch.tensor(values_tokens[0]).unsqueeze(0)))

In [55]:
model.electra(torch.tensor(query_tokens).unsqueeze(0))

C:\Users\stani\AppData\Local\Temp\ipykernel_14692\1773758331.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model.electra(torch.tensor(query_tokens).unsqueeze(0))


BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[ 5.4972e-01, -4.0423e-04, -1.4782e-04,  ..., -1.1460e-03,
           7.1502e-01,  2.2931e-04],
         [ 1.9972e+00, -7.0433e-04, -1.8687e-04,  ..., -1.9971e-04,
           4.7837e-01,  1.1312e-03],
         [ 8.4307e-01, -3.8002e-04, -6.8160e-05,  ...,  7.0667e-04,
           7.3262e-01,  3.1652e-05],
         [ 5.7297e-01, -2.7322e-04, -1.7928e-04,  ..., -8.8661e-04,
           1.7585e-02,  2.7722e-04],
         [ 5.4900e-01, -4.0361e-04, -1.4781e-04,  ..., -1.1452e-03,
           7.1440e-01,  2.2975e-04]]], grad_fn=<NativeLayerNormBackward0>), past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)

In [ ]:
torch.sort(result)

In [ ]:
with open('result.txt', 'w', encoding='utf-8') as file:
    for index in torch.sort(result, descending=True)[1]:
        file.write('___---___---___\n')
        file.write(data[index])